In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-09-15 15:48:20--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.3’

minsearch.py.3      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-09-15 15:48:20 (13.2 MB/s) - ‘minsearch.py.3’ saved [3832/3832]



In [2]:
import minsearch

In [3]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-09-15 15:48:21--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json.3’

documents.json.3    100%[===================>] 642.90K  3.15MB/s    in 0.2s    

2024-09-15 15:48:21 (3.15 MB/s) - ‘documents.json.3’ saved [658332/658332]



In [4]:
import json

In [5]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [6]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [7]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [8]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [9]:
q = 'the course has already started, can I still enroll?'

In [10]:
index.fit(documents)

In [11]:
boost = {
    "question": 3.0,
    "section": 0.5
}


results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [13]:
from openai import OpenAI

In [14]:
client = OpenAI()

In [16]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION : {question}

CONTEXT : {context}
""".strip()

In [17]:
context = ""

for doc in results:
    context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [18]:
prompt = prompt_template.format(question=q, context=context).strip()

In [20]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[{'role': 'user', 'content': prompt}]
)

response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

### The RAG Flow Cleaning and Modularizing Code

In [1]:
import json
import minsearch

from openai import OpenAI

In [2]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

client = OpenAI()

In [3]:
def search(query):
    boost = {
        "question": 3.0,
        "section": 0.5
    }
    
    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [4]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION : {question}
    
    CONTEXT : {context}
    """.strip()

    context = ""

    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    return prompt_template.format(question=query, context=context).strip()

In [5]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{'role': 'user', 'content': prompt}]
    )
    
    return response.choices[0].message.content

In [6]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    return llm(prompt)

In [7]:
query = 'how do I run Kafka?'
answer = rag(query)
print(answer)

To run Kafka, follow these steps based on your programming language:

1. **For Java:**
   In your project directory, run the following command in the terminal:
   ```bash
   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java
   ```

2. **For Python:**
   First, create a virtual environment and install the necessary dependencies. You can do this by executing the following commands:
   ```bash
   python -m venv env
   source env/bin/activate  # For MacOS/Linux
   # For Windows, use: env\Scripts\activate
   pip install -r ../requirements.txt
   ```
   Remember to activate the virtual environment each time you want to run your Python files.

Make sure all Docker images are up and running if you're using Docker.


In [8]:
rag('the course already started, can I still enroll?')

'Yes, you can still enroll in the course even after it has started. You are eligible to submit homework assignments, but keep in mind that there will be deadlines for turning in the final projects, so it’s best not to leave things until the last minute.'